In [1]:
# some experiments
import networkx as nx
import numpy as np
import pandas as pd
# from utils.problem_builder import generate_graph

In [56]:
# import data
orders_data = pd.read_csv('./orders.csv')[:6]
distance_matrix = pd.read_csv('./distance_matrix.csv', index_col=0).iloc[:6, :6]


In [57]:
distance_matrix

,Kraków,Białystok,Bielsko-Biała,Chrzanów,Gdańsk,Gdynia
Kraków,0.000000,477.395329,102.318127,59.955709,475.243743,492.744242
Białystok,477.395329,0.000000,570.589737,518.682870,502.064756,532.182685
Bielsko-Biała,102.318127,570.589737,0.000000,52.024945,485.320986,497.323453
Chrzanów,59.955709,518.682870,52.024945,0.000000,455.486086,469.958469
Gdańsk,475.243743,502.064756,485.320986,455.486086,0.000000,30.144525
Gdynia,492.744242,532.182685,497.323453,469.958469,30.144525,0.000000


In [58]:
def generate_graph(orders_data, distance_matrix):
    G = nx.Graph()
    for city in orders_data['city']:
        row = orders_data[orders_data['city'] == city]
        location = (row['longitude'].iloc[0], row['latitude'].iloc[0])
        G.add_node(city, location=location)

    for origin in orders_data['city']:
        for destination in orders_data['city']:
            if origin != destination:
                distance = distance_matrix.at[origin, destination]
                G.add_edge(origin, destination, distance=distance)
    
    return G


G = generate_graph(orders_data, distance_matrix)

In [59]:
print(G.edges)

[('Kraków', 'Białystok'), ('Kraków', 'Bielsko-Biała'), ('Kraków', 'Chrzanów'), ('Kraków', 'Gdańsk'), ('Kraków', 'Gdynia'), ('Białystok', 'Bielsko-Biała'), ('Białystok', 'Chrzanów'), ('Białystok', 'Gdańsk'), ('Białystok', 'Gdynia'), ('Bielsko-Biała', 'Chrzanów'), ('Bielsko-Biała', 'Gdańsk'), ('Bielsko-Biała', 'Gdynia'), ('Chrzanów', 'Gdańsk'), ('Chrzanów', 'Gdynia'), ('Gdańsk', 'Gdynia')]


In [60]:
import itertools
def calculate_total_distance(G, route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += G[route[i]][route[i+1]]['distance']
    return total_distance

cities = list(G.nodes)
shortest_distance = float('inf')
shortest_route = None

for route in itertools.permutations(cities[1:]):
    route = [cities[0]] + list(route) + [cities[0]]  # Start and end at the first city
    distance = calculate_total_distance(G, route)
    if distance < shortest_distance:
        shortest_distance = distance
        shortest_route = route

print("Shortest Route:", shortest_route)
print("Total Distance:", shortest_distance)


Shortest Route: ['Kraków', 'Chrzanów', 'Bielsko-Biała', 'Gdynia', 'Gdańsk', 'Białystok', 'Kraków']
Total Distance: 1618.908717205671


In [61]:
def nearest_neighbor(G, depot, unvisited):
    route = [depot]
    current_city = depot

    while unvisited:
        next_city = min(unvisited, key=lambda x: G[current_city][x]['distance'])
        route.append(next_city)
        unvisited.remove(next_city)
        current_city = next_city

    route.append(depot)
    return route

In [69]:
depot = 'Kraków'  # Replace with your depot city
cities = list(G.nodes)
cities.remove(depot)

route = nearest_neighbor(G, depot, cities.copy())
print("Route:", route)
print("Distance:", calculate_total_distance(G, route))

Route: ['Kraków', 'Chrzanów', 'Bielsko-Biała', 'Gdańsk', 'Gdynia', 'Białystok', 'Kraków']
Distance: 1637.024179318168


In [65]:
# def simulated_annealing(G, initial_solution, initial_temp, final_temp, alpha):
#     current_solution = initial_solution
#     current_cost = calculate_cost(G, current_solution)
#     temperature = initial_temp

#     while temperature > final_temp:
#         neighbor = generate_neighbor(current_solution)
#         neighbor_cost = calculate_cost(G, neighbor)
        
#         if acceptance_probability(current_cost, neighbor_cost, temperature) > random.random():
#             current_solution = neighbor
#             current_cost = neighbor_cost

#         temperature *= alpha  # Cooling down

#     return current_solution

# def acceptance_probability(current_cost, neighbor_cost, temperature):
#     if neighbor_cost < current_cost:
#         return 1.0
#     return math.exp((current_cost - neighbor_cost) / temperature)

# # Example parameters
# initial_temp = 10000
# final_temp = 1
# alpha = 0.99

# # Generate an initial solution
# initial_solution = generate_initial_solution(G, num_vehicles)

# # Run the algorithm
# solution = simulated_annealing(G, initial_solution, initial_temp, final_temp, alpha)

In [70]:

depot = 'Depot'  # Replace with your depot city
cities = list(G.nodes)
cities.remove(depot)
num_vehicles = 2  # Number of vehicles

routes = vrp_nearest_neighbor(G, depot, cities, num_vehicles)
for i, route in enumerate(routes):
    print(f"Vehicle {i+1} Route:", route)

ValueError: list.remove(x): x not in list

In [68]:
G.nodes

NodeView(('Kraków', 'Białystok', 'Bielsko-Biała', 'Chrzanów', 'Gdańsk', 'Gdynia'))